# this notebook shows backtestlib.optimizer.BenchMark etc.

In [1]:
import sys
sys.path.insert(0, "..")

import logging as log
log.basicConfig(level = log.INFO, stream=sys.stdout,\
                format='backtest-%(asctime)s-%(funcName)s: %(message)s', \
                datefmt='%Y-%b-%d %H:%M:%S')
from  backtestlib.optimizer import BenchMark, SampleBased, SpectralCut, SpectralSelection,ShrinkToIdentity,POET
import pandas as pd
import numpy as np
import numpy.linalg as LA
%load_ext autoreload
%autoreload 2

In [2]:
signal = pd.Series(data = [1,2,2,1.5,-.2],index = ['AAPL','GOOGLE','MMM','BAC','GS'])
signal

AAPL      1.0
GOOGLE    2.0
MMM       2.0
BAC       1.5
GS       -0.2
dtype: float64

In [3]:
c = 0.02
r = 0.00
n = 50
p = len(signal)

rand = np.random.RandomState(1233)
true_var = np.diag(np.ones(p))*(1-c) + c*np.ones((p,p))
true_mu = np.ones(p)*r

returns = rand.multivariate_normal(true_mu, true_var, n).T
sample_var =(returns@returns.T)/n

In [4]:
opt = BenchMark()
opt.optimize(signal = signal)

backtest-2021-Feb-22 01:54:59-__init__: BenchMark initialized


array([0.2, 0.2, 0.2, 0.2, 0.2])

In [5]:
opt = SampleBased()
opt.optimize(signal = signal,sample_var = sample_var)

backtest-2021-Feb-22 01:54:59-__init__: SampleBased initialized


array([ 0.13203564,  0.36667792,  0.40506277,  0.16931761, -0.07309393])

In [6]:
opt = SpectralCut(delta = 0.4)
opt.optimize(signal = signal,sample_var = sample_var)

backtest-2021-Feb-22 01:54:59-__init__: SpectralCut initialized with  delta 0.4
backtest-2021-Feb-22 01:54:59-optimize: Optimal K is 5
backtest-2021-Feb-22 01:54:59-optimize:  return component [-0.56292848  0.56166544 -1.79661716 -0.34753213  2.70352208]


array([ 0.13203564,  0.36667792,  0.40506277,  0.16931761, -0.07309393])

In [7]:
opt = ShrinkToIdentity()
opt.optimize(signal = signal,returns = returns)

backtest-2021-Feb-22 01:54:59-__init__: ShrinkToIdentity initialized
backtest-2021-Feb-22 01:54:59-optimize: shrink coefficients: a 1.708, b 0.3, d 1.734, m 0.858


array([ 0.1141222 ,  0.40128602,  0.42972691,  0.16964652, -0.11478166])

In [8]:
opt = SpectralSelection(delta = 0.4, c = 10)
opt.optimize(signal = signal,sample_var = sample_var)

backtest-2021-Feb-22 01:54:59-__init__: SpectralCut initialized with  delta 0.4, c 10
backtest-2021-Feb-22 01:54:59-optimize: Optimal gamma is 0.003
backtest-2021-Feb-22 01:54:59-optimize:  return component [-0.56266804  0.55608824 -1.7810237  -0.2796166   1.36131824]


array([ 0.16811186,  0.28152213,  0.30907235,  0.28438214, -0.04308848])

- To demostrate POET we simulate returns via a factor model of K factors such that K << p~n
- K: number of factors, p asset numbers, n time periods
- 

In [9]:
c = 0.3
n = 200
p = 100

rand = np.random.RandomState(1234)
temp = rand.normal(size = (p,n))
temp_u,temp_s,temp_vh =  LA.svd(temp)

true_K = 6
true_B = temp_u[:,:true_K]@np.diag(temp_s[:true_K])
true_F = temp_vh.T[:,:true_K]
errors = rand.normal(size = (p,n))*c
returns = true_B@true_F.T + errors

signal = pd.Series(np.ones(p))

In [13]:
opt = POET(C = 0.2)
opt.optimize(signal = signal,returns = returns)

backtest-2021-Feb-22 02:00:12-__init__: ShrinkToIdentity initialized
backtest-2021-Feb-22 02:00:12-optimize: Optimal K is 6
backtest-2021-Feb-22 02:00:12-optimize: offdiagnal f-norm under C=0.2 is 0.0531


array([0.01390297, 0.00604776, 0.007464  , 0.00991726, 0.01035616,
       0.0077557 , 0.01307507, 0.00813021, 0.00889522, 0.0100044 ,
       0.00909759, 0.0117812 , 0.00921658, 0.00972147, 0.00849731,
       0.00739705, 0.01212967, 0.00831121, 0.01300866, 0.00815562,
       0.01147961, 0.00915036, 0.01670194, 0.00795584, 0.0124574 ,
       0.01477995, 0.01013923, 0.01040963, 0.00567854, 0.01220117,
       0.01081098, 0.01016651, 0.00899532, 0.01096248, 0.00952144,
       0.00741788, 0.01364388, 0.00842481, 0.01177619, 0.01321186,
       0.00655681, 0.01252246, 0.01187519, 0.00956167, 0.0077062 ,
       0.00878962, 0.01045418, 0.01316003, 0.0083652 , 0.00747967,
       0.00536865, 0.01260353, 0.00965487, 0.00721568, 0.01363581,
       0.01386676, 0.01453695, 0.01153453, 0.00977448, 0.00709495,
       0.00851572, 0.00876866, 0.0083318 , 0.00653354, 0.00771605,
       0.01110796, 0.00927917, 0.00846805, 0.00669602, 0.01434645,
       0.01295281, 0.01070698, 0.01029886, 0.0103633 , 0.00539